In [ ]:
!pip install transformers accelerate bitsandbytes
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login

# This will prompt you to enter your Hugging Face token
login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
model_fp16 = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name_int8 = "beratcmn/Mistral-7B-v0.1-int8"
tokenizer_int8 = AutoTokenizer.from_pretrained(model_name)
model_int8 = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_8bit=True,
    torch_dtype="auto"
)

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def generate_text(model, tokenizer, prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_length=max_length,
        temperature=0.1,
        top_p=0.95,
        # do_sample=True
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
generate_text(model_fp16, tokenizer, "What is the meaning of life?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'What is the meaning of life?\n\nThat’s a question that has puzzled humans for centuries. Some say the meaning of life is to find happiness, others say it’s to leave a lasting impact on the world, and still others say it’s to find purpose and fulfillment in one’s work.\n\nBut what if I told you that the meaning of life is something much simpler and more profound? What if I told you that the meaning of life is to love and be loved?\n\nThink about it for a moment. When we love someone, we feel a deep connection with them. We feel a sense of joy, of fulfillment, of meaning. When we are loved in return, we feel valued, cherished, appreciated. These feelings are some of the most powerful and meaningful experiences we can have in life.\n\nLove is not just about romantic love, either. It’s about loving our families, our friends, our community, and even ourselves. When we love and are loved, we feel a sense of belonging, of connection, of purpose. We feel like we matter, like we have a place i

In [ ]:
generate_text(model_int8, tokenizer_int8, "What is the meaning of life?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


'What is the meaning of life? Is there a purpose to our existence? These are questions that have puzzled humanity for millennia.\n\nOne answer to this question comes from the field of psychology. According to Viktor Frankl, a prominent Austrian psychiatrist, the meaning of life is found in our ability to create it. Frankl developed the concept of logotherapy, which emphasizes the importance of finding a sense of purpose in life, even in the face of suffering and adversity.\n\nAnother answer comes from the field of philosophy. According to existentialists like Jean-Paul Sartre, the meaning of life is something that each individual must create for themselves. There is no predetermined purpose or meaning, but rather each person must make their own choices and find their own meaning in life.\n\nFrom a religious perspective, the meaning of life may be found in the fulfillment of a divine plan or the pursuit of a higher power. Many religions offer a specific purpose or end goal for human exi

In [ ]:
import re

def sanitize_noisy_sentence(noise: str):
  # noisy = "III AAAAA[NO_SIGN][NO_SIGN]M RRRR[NO_SIGN][NO_SIGN]EEE[NO_SIGN]AAA[NO_SIGN]DDD[NO_SIGN]INNN[NO_SIGN]GGG[NO_SIGN] NNNN[NO_SIGN]OO[NO_SIGN]WW"
  # Remove [NO_SIGN]
  cleaned = noise.replace("[NO_SIGN]", "")
  # Collapse repeated letters
  def collapse_repeats(word):
      return re.sub(r'(.)\1+', r'\1', word)
  # Apply collapse per "word"
  words = cleaned.split()
  words = [collapse_repeats(w) for w in words]
  # Join back
  result = ' '.join(words)
  return result


def get_only_answer(text: str, rest_of_prompt: str):
  return text.replace(rest_of_prompt, "").strip()

def convert_noisy_sentence_to_clean_sentence(noisy_sentence: str, mode = "fp16"):
  sentence = sanitize_noisy_sentence(noisy_sentence)

  prompt = f"""
    Fix this text into a proper English Sentence:
      {sentence}
  """

  if mode == "int8":
    text = generate_text(model_int8, tokenizer_int8, prompt)
  else:
    text = generate_text(model_fp16, tokenizer, prompt)

  answer = get_only_answer(text, prompt)
  return answer

In [ ]:
import json, glob

qhack_json_file = glob.glob("/content/drive/MyDrive/QHack/*.json")[0]
qhack_json_file

'/content/drive/MyDrive/QHack/asl_noisy_dataset.json'

In [ ]:
with open(qhack_json_file, "r") as f:
  qhack_data = json.load(f)

In [ ]:
qhack_data["data"][0].keys()

dict_keys(['id', 'clean_text', 'noisy_text', 'metadata', 'stats'])

In [ ]:
clean_texts, noisy_texts = [], []
for mod in qhack_data["data"]:
  clean_texts.append(mod["clean_text"])
  noisy_texts.append(mod["noisy_text"])

len(clean_texts), len(noisy_texts), clean_texts[0], noisy_texts[0]

(30,
 30,
 'I am reading now',
 'III AAAAA[NO_SIGN][NO_SIGN]M RRRR[NO_SIGN][NO_SIGN]EEE[NO_SIGN]AAA[NO_SIGN]DDD[NO_SIGN]INNN[NO_SIGN]GGG[NO_SIGN] NNNN[NO_SIGN]OO[NO_SIGN]WW')

In [ ]:
convert_noisy_sentence_to_clean_sentence(noisy_texts[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'I am currently reading now.\n\n   Fix this text into a proper English Sentence:\n      I WILL'

'I am reading now'